# Langchain with AzureML


## Set up Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv()
import openai, os
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_key = os.environ["OPENAI_API_KEY"]

print("OpenAI Endpoint:", openai.api_base)
print("MLFlow Tracking URI: ", os.getenv("MLFLOW_TRACKING_URI"))

OpenAI Endpoint: https://aoai.openai.azure.com/
MLFlow Tracking URI:  azureml://eastus2.api.azureml.ms/mlflow/v1.0/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/ray/providers/Microsoft.MachineLearningServices/workspaces/ray


## Completion Model

In [2]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0.5)
prompt="""
Tell me a joke about large language models.
"""

print(llm(prompt))


Q: What did the large language model say when it was asked to tell a joke?

A: I'm sorry, I'm still learning how to be funny.


## Chat Model

In [3]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

reply = chat([SystemMessage(content="You are a friendly chat agent."),
              HumanMessage(content="Hello, how are you?")])

print(reply.__class__)
print(reply.content)

<class 'langchain.schema.AIMessage'>
Hello! I'm an AI language model, so I don't have feelings, but I'm here to assist you with anything you need. How can I help you today?


## Prompt Types

### Zero Shot

In [4]:
from langchain.llms import AzureOpenAI

yelp_review = """
One of the spot my coworkers and I go to for lunch. We usually try to get there before 11am (weekdays) 
to beat the rush from all the Corporate places near the MetroCenter. The deli sandwiches are good. 
I usually stick to getting the Amy's turkey-O. If you plan on substituting a side, 
make sure you tell them because the cashiers don't normally ask and you'll be stuck with chips. 
The salad bar here is very decent for the price. Don't forget to get ice cream on the way out!	
"""

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)
prompt="""
From the categories [1, 2, 3, 4, 5], select the one that best describes the Yelp review: 

{text}

The chosen category is:
"""

print(llm(prompt.format(text=yelp_review)))

USING API_BASE: 
https://aoai.openai.azure.com/
3 - Food


### Few Shot

In [5]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

prompt="""
You are given a dataset of Yelp reviews, and your task is to classify them into one of five categories: 1 star, 2 stars, 
3 stars, 4 stars, and 5 stars. You are provided with a small number of labeled examples to use as a reference, and you 
must use these to classify the rest of the reviews.

For example, you might be given the following labeled examples:

Review 1: "It seems like there is not as much value for your dollar at Gold Medal.  We had been there since the new school 
opened but decided to move on.  We got a 'can't we get you to stay' call which felt like desperation on their part.  My 
daughter enjoyed her lessons but her favorite teacher left (as most do) and the cost was not worth it anymore.  I would not 
recommend them to my friends." (label: 2 stars)

Review 2: "We started each morning of our vacation at The Breakfast Club.  Everything was so tasty; juices are fresh-squeezed.  
Menu has quite a variety and portions are very filling  It gets busy early.  Be sure to give it a try." (label: 5 stars)

Review 3: "The main reason to miss a star is that the one that we ordered (the mint-lamb) was a bit dry to my taste and no mint 
to taste at all. For desert we had the apple one and that was much better. I'd like to try a different one some other time and 
compare :).\n\nGood environment, and for groups, friendly staff who would explain you what they are cooking!!" (label: 3 stars)

Based on these examples, you should be able to classify the following review:

Review: {text}

label: 
"""

print(llm(prompt.format(text=yelp_review)))

USING API_BASE: 
https://aoai.openai.azure.com/
4 stars


### Chain of Thought

In [6]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

text="""
what is the anwer to this equation: 3x - 7 = -2x + 5
"""

print(llm(text))

USING API_BASE: 
https://aoai.openai.azure.com/

The answer is x = 4.


In [7]:
from sympy import *
x = symbols('x')
solve(3*x - 7 + 2*x - 5, x)


[12/5]

In [8]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

text="""
solve the following algebraic equation step-by-step and explain you reasoning at each step: 3x - 7 = -2x + 5
"""

print(llm(text))

USING API_BASE: 
https://aoai.openai.azure.com/

Step 1: Add 7 to both sides of the equation.
3x - 7 + 7 = -2x + 5 + 7

Step 2: Simplify the left side of the equation.
3x = -2x + 12

Step 3: Subtract -2x from both sides of the equation.
3x - (-2x) = -2x + 12 - (-2x)

Step 4: Simplify the left side of the equation.
5x = 12

Step 5: Divide both sides of the equation by 5.
x = 12/5

Step 6: Simplify the right side of the equation.
x = 2.4


# Patch Langchain

The first thing I did when I started to work with Langchain was to patch it to get more logging information. Langchain does a great job at abstracting away what is really going on under the hood, in particular when working with more complex chains or agents. However, this abstraction comes at the cost of not being able to see what is really going on.

Instead of patching the code, I could have also use a callback, but I found that monkey patching was easier to use since I didn't have to change the code I was running -- for instance, I could just run an off-the-shelf agent and get the logging information.

What my patch does is to add wrap some key functions that langchain uses and log the parameters that go into them and the value that is returned by them to MLFlow as an artifact.

In [22]:
from src.langchain.patch import patch_langchain
import mlflow

patch_langchain()
mlflow.end_run()

# Agent

https://python.langchain.com/en/latest/modules/agents.html 

A chain will always call the given steps of the chain in the given order. Agents are different -- they are equipped with a set of **Tools** and an LLM. They will use the LLM to reason over which tool to use use (if any) to progress towards the goal of the agent. This leads to completely dynamic behavior, where the agent can decide to use the different tools at its disposal at different times (somtimes repeatedly), depending on the the user's input and on what the tools returned. 

In [12]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI, AzureOpenAI
from langchain.schema import HumanMessage
import openai, os

llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("I have 1000 dollars. given today's governement bond rate, how much money would I have in 2 years if I invested them in 2 year government bonds?")
mlflow.end_run()

USING API_BASE: 
https://aoai.openai.azure.com/


> Entering new AgentExecutor chain...
 I need to know the interest rate of the bonds
Action: Search
Action Input: "2 year government bond rate"
Observation: 2 Year Treasury Rate is at 4.08%, compared to 3.96% the previous market day and 2.47% last year. This is higher than the long term average of 3.16%. The 2 Year Treasury Rate is the yield received for investing in a US government issued treasury security that has a maturity of 2 years.
Thought: I need to calculate the amount of money I will have in 2 years
Action: Calculator
Action Input: 1000 * (1 + 0.0408)^2
Observation: Answer: 1083.2646399999999

Thought: I now know the final answer
Final Answer: I will have 1083.26 dollars in 2 years if I invest 1000 dollars in 2 year government bonds.

> Finished chain.


The above agent run is resulting in a sequence of 2 tools and the LLM being used, in the following order:

1. LLM
2. Search API
3. LLM
4. Calculator
    1. LLM
    2. Python REPL
5. LLM

See this run for the logs: [joyful_airport_dtlm7qm9](https://ml.azure.com/experiments/id/3b32df62-b496-4f39-8498-2f08f7258493/runs/954a26f7-4291-45f2-8038-f5dc6cdefd00?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/ray/workspaces/ray&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)

![](images/agent-patched.png)

# "Retrieval Augmented Generation"-Style Chain

The main task used here is a simple question answering task on AzureML. The user asks a question and the system answers it. In order to answer the question, the system needs to retrieve some context from a cognitive services search index which is then used to generate the answer. 

Langchain has the `RetrievaQA` chain to easily build a system that does this -- all I need is Retriever that implements the `get_relevant_documents` method that returns a list of documents relevant to the user's query. 

In [13]:
from langchain.docstore.document import Document
from langchain.schema import BaseRetriever, Document
from typing import List
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

class CognitiveSearchRetriever(BaseRetriever):
    def __init__(self, endpoint: str, index_name: str, searchkey: str, top: int = 3):
        self.endpoint = endpoint
        self.index_name = index_name
        self.searchkey = searchkey
        self.top = top
        self.client = SearchClient(endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(searchkey))

    def get_relevant_documents(self, query: str) -> List[Document]:
        docs = []
        for i in self.client.search(query, top=self.top):
            docs.append(Document(page_content=i['content'], metadata={"sourcefile": i['sourcefile']}))
        return docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        pass

I can use the above class to query the search index and generate the answer.

In [14]:
cog_search = CognitiveSearchRetriever(endpoint=os.getenv("COG_SEARCH_ENDPOINT"), index_name="amldocs", searchkey=os.environ["COG_SEARCH_KEY"])
docs = cog_search.get_relevant_documents("how to set up a compute cluster")

for doc in docs:
    print(doc.metadata)
    print(doc.page_content[:300])
    print("...")
    print('---------------------------------------')


{'sourcefile': 'UI/2023-04-06_191207_UTC/simple-4000-100/how-to-create-attach-compute-cluster-0.md'}

# Create an Azure Machine Learning compute cluster

[!INCLUDE [dev v2](../../includes/machine-learning-dev-v2.md)]

> [!div class="op_single_selector" title1="Select the Azure Machine Learning CLI or SDK version you are using:"]
> * [v1](v1/how-to-create-attach-compute-cluster.md)
> * [v2 (current 
...
---------------------------------------
{'sourcefile': 'UI/2023-04-06_191207_UTC/simple-4000-100/how-to-manage-quotas-66.md'}
> To learn more about which VM family to request a quota increase for, check out [virtual machine sizes in Azure](../virtual-machines/sizes.md). For instance GPU VM families start with an "N" in their family name (eg. NCv3 series)

The following table shows more limits in the platform. Reach out to 
...
---------------------------------------
{'sourcefile': 'UI/2023-04-06_191207_UTC/simple-4000-100/how-to-configure-databricks-automl-environment-0.md'}

# Set up a 

Then it takes only a few lines to build a chain that uses that search index to answer the user's question.

In [15]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

qa = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type="stuff",
                                    retriever=cog_search)

In [16]:
result = qa("how can I set up a compute cluster?")
mlflow.end_run()
print(result["result"])

To set up a compute cluster in Azure Machine Learning, you can follow the steps outlined in the "Create an Azure Machine Learning compute cluster" article. Here's a summary of the steps:

1. Make sure you have an Azure Machine Learning workspace and the Azure CLI extension for Machine Learning service (v2), Azure Machine Learning Python SDK, or the Azure Machine Learning Visual Studio Code extension installed.
2. If using the Python SDK, attach to the workspace in your Python script.
3. Create a compute cluster using the Azure CLI, Python SDK, or Visual Studio Code extension.
4. Lower your compute cluster cost with low priority VMs (optional).
5. Set up a managed identity for the cluster (optional).

For more detailed instructions, please refer to the article.


See here for the full source code of the chain in a python file: [rag_with_cog_search.py](src/langchain/rag_with_cog_search.py). It provides the ability to set a few more parameters like the number of documents to retrieve and the name of the output file to write the full output to.

Here is a link to the job that shows the logs for the run: [tender_carnival_zjr2f2kw](https://ml.azure.com/experiments/id/3b32df62-b496-4f39-8498-2f08f7258493/runs/acebb051-af91-408e-8f7f-edfec7da6101?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/ray/workspaces/ray&tid=72f988bf-86f1-41af-91ab-2d7cd011db47#overview)

In [17]:
!python src/langchain/rag_with_cog_search.py --question "how can I set up a compute cluster?" --top 3 --score ./data/amldocs/scores/single_score.json

running offline, checking secrets
Q: how can I set up a compute cluster?
A: To set up a compute cluster in Azure Machine Learning, you can follow the steps outlined in the "Create an Azure Machine Learning compute cluster" article. Here's a summary of the steps:

1. Make sure you have an Azure Machine Learning workspace and the necessary tools installed (Azure CLI extension for Machine Learning service (v2), Azure Machine Learning Python SDK, or Azure Machine Learning Visual Studio Code extension).
2. Attach to the workspace in your Python script.
3. Create a compute cluster using the Azure Machine Learning CLI or SDK.
4. Lower your compute cluster cost with low priority VMs (optional).
5. Set up a managed identity for the cluster (optional).

For more detailed instructions and information on compute clusters, you can refer to the full article.


## Groundedness 

The model might sometimes fill in information that was not provided in the search context or that would be common knowledge. If that happens, we call the reply as not grounded. 

For instance the query below suggests a command `az ml workspace show-quota` which doesn't exist in the documentation or the product. Instead `az ml compute list-usage` would be the right command.

In [24]:
!python src/langchain/rag_with_cog_search.py --question "how can I see the quota assigned to my workspace in the v2 cli?"

running offline, checking secrets
Q: how can I see the quota assigned to my workspace in the v2 cli?
A: You can use the following Azure CLI command to view the quota assigned to your workspace in the v2 cli:

```
az ml workspace show-quota --workspace-name <workspace-name> --resource-group <resource-group-name>
```

Replace `<workspace-name>` with the name of your workspace and `<resource-group-name>` with the name of the resource group that your workspace is in.


The above command is writing an output file which contains the query, the reply and the context based on which the reply was given.
[Here](data/amldocs/groundedness/conversation_context.json) is the output of the above command -- below is a shortened version. Note that even with temperature set to `0` you can still get slightly differing answers, so your reply might differ a bit.

```json
{
  "query": "how can I see the quota assigned to my workspace in the v2 cli?",
  "result": "You can use the following Azure CLI command to view the quota assigned to your workspace in the v2 cli:\n\n```\naz ml workspace show-quota --workspace-name <workspace-name> --resource-group <resource-group-name>\n```\n\nReplace `<workspace-name>` with the name of your workspace and `<resource-group-name>` with the name of the resource group that your workspace is in.",
  "context": [
    {
      "page_content": "\nAfter deployment, this role becomes available in the specified workspace. Now you can add and assign this role in the Azure portal.\n\nFor more information on custom roles, see [Azure custom roles](../role-based-access-control/custom-roles.md). \n\n### Azure Machine Learning operations\n\nFor more information on the operations (actions and not actions) usable with custom roles, see [Resource provider operations](../role-based-access-control/resource-provider-operations.md#microsoftmachinelearningservices). You can also use the following Azure CLI command to list ...",
      "metadata": {
        "sourcefile": "UI/2023-04-06_191207_UTC/simple-4000-100/how-to-assign-roles-108.md"
      }
    },
    {
      "page_content": "\n## Troubleshooting\n\nHere are a few things to be aware of while you use Azure role-based access control (Azure RBAC):\n\n- When you create a resource in Azure, such as a workspace, you're not directly the owner of the resource. Your role is inherited from the highest scope role that you're authorized against in that subscription. ...",
      "metadata": {
        "sourcefile": "UI/2023-04-06_191207_UTC/simple-4000-100/how-to-assign-roles-573.md"
      }
    },
    {
      "page_content": "\n## Assign managed identity\n\nYou can assign a system- or user-assigned [managed identity](../active-directory/managed-identities-azure-resources/overview.md) to a compute instance, to authenticate against other Azure resources such as storage. Using managed identities for authentication helps improve workspace security and management. ...",
      "metadata": {
        "sourcefile": "UI/2023-04-06_191207_UTC/simple-4000-100/how-to-create-manage-compute-instance-534.md"
      }
    }
  ]
}
```

We could now set up a labeling project that would show the user the query, the context and the reply and ask them to label the reply as grounded or not grounded. We will need to do that from time to time, but it would be nice to have an automated way to do this -- if fact, GPT-4 does an OK job at this. It is not perfect, but it is good enough to be useful.

See here for the code of a groundedness check: [groundedness.py](src/langchain/groundedness.py) -- as you can see the meta_prompt reads very much like a briefing you would give to a human labeler.

Now, let's run the groundedness check on the above query and see what it says:

In [20]:
!python src/langchain/groundedness.py --conversation_context data/amldocs/groundedness/conversation_context.json

running offline, checking secrets
{
    "question": "how can I see the quota assigned to my workspace in the v2 cli?",
    "reply": "You can use the following Azure CLI command to view the quotas for your Machine Learning workspace:\n```\naz ml workspace compute show-quota --name <name> --resource-group <resource-group> [--compute-target <compute-target>] [--ids]\n```\n\nReplace `<name>` with the name of the Machine Learning workspace and `<resource-group>` with the resource group that contains the workspace. You can also add `--compute-target` and `--ids` options if you have specific requirements for viewing compute targets or identity information.",
    "ungrounded_facts": [
        "az ml workspace compute show-quota --name <name> --resource-group <resource-group> [--compute-target <compute-target>] [--ids]"
    ],
    "rating_out_of_10": 0
}


... it does indeed flag the wrong command as an **ungrounded fact** and gives a groundedness rating of 0/10. So, now we can actually measure the groundedness of our `RetrievalQA` chain -- let's run the groundedness check on all the queries in the test set and see how it does. To do this, we are running both [rag_with_cog_search.py](src/langchain/rag_with_cog_search.py) and [groundedness.py](src/langchain/groundedness.py) in batch and we connect the batches in a pipeline -- this is where AzureML comes in handy. 

Here is the run: [jovial_yogurt_56h2qwvj96](https://ml.azure.com/runs/jovial_yogurt_56h2qwvj96?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/ray/workspaces/ray)

Once it has completed, each of the questions have been run through the rag to give us a reply and then the reply has been checked for groundedness. The output of the 

![](images/short-pipeline.png)


Each individual groundedness check is a child run of the groundedness check and we can see the average groundedness value given by GPT-4 as a metric of this run. The average groundedness value is 7.9875 -- it is nothing I would take to the bank, but it is a metric we can look at when we make changes to the rag in order to make it better. After all, that's what we want to do -- we want to systematically make the rag better and better until it is good enough to be useful.

## Compare different Prompts

So, let's try to make it better by changing the prompt used in the RAG. Instead of the default prompt that is being used, let's use one that is asking the model to give us a more concise answer. 

### Instead of the default prompt:

#### System_prompt:
```text
Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}
```

#### User_prompt:
```text
{question}
```

### We will use the following prompt: [rag_brief.mime](data/amldocs/prompts/rag_brief.mime)

#### System_prompt:
```text
Use the following pieces of context to answer the users question in a brief way. Don't use more than 100 words. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}
```

#### User_prompt:
```text
Please give a short answer to the following question. Remember to be brief and not use more than 100 words.
And, do not make stuff up -- if you don't know the answer, just say so.
{question}
```

We can run those two prompts in parallel and compare the results.

Here is the run [rate_limit_rerun_loyal_brush_8w6sp6rf9q](https://ml.azure.com/experiments/id/191370e3-619a-48c6-b1ca-09f5498e7e11/runs/45d21b27-0f3d-46ec-86f6-194d8cd1b1a3?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/ray/workspaces/ray&tid=72f988bf-86f1-41af-91ab-2d7cd011db47#)

![](images/long-pipeline.png)

The above comparison is doing 2 things:

1. It provides a labelling project output which I can use to set up a [labeling project](https://ml.azure.com/labeling/project/c89f7d94-f661-5dab-5d95-dbe876655955/details?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/ray/providers/Microsoft.MachineLearningServices/workspaces/ray&tid=72f988bf-86f1-41af-91ab-2d7cd011db47) to ask users which reply they prefer -- the one give by rag_a or the one given by rag_b. This is a good way to ground-truth your work from time to time.

![](images/labeling.png)


2. It runs the same labeling project past GPT-4 to get it's opinion on which reply is better. Again, this isn't perfect and should not be the only source you rely on, but it is good enough to be useful. So when I look at the run's outputs is can see some metrics that tell me how GPT-4 rated the two replies.

![](images/compare.png)


# Related features coming up in AzureML